情绪分析(emotional analysis)。它和情感分析(sentiment analysis)有相似之处。都是通过对内容的自动化分析，来获得结果。

情感分析的结果一般分为正向(positive)和负向(negative)，而情绪分析包含的种类就比较多了。

加拿大国家研究委员会(National Research Council of Canada)官方发布的情绪词典包含了8种情绪，分别为：

愤怒(anger)
期待(anticipation)
厌恶(disgust)
恐惧(fear)
喜悦(joy)
悲伤(sadness)
惊讶(surprise)
信任(trust)
有了这些情绪的标记，你可以轻松地对一段文本的情绪变化进行分析。

这时候，你可以回忆起中学语文老师讲作文时说过的那句话：

文如看山不喜平。

故事情节会伴随着各种情绪的波动。通过分析这些情绪的起伏，我们可以看出故事的基调是否符合自己的口味，情节是否紧凑等。这样，你可以根据自己的偏好，甚至是当前的心境，来选择合适的作品观看了。

**我们需要用到Python和R。这两种语言在目前数据科学领域里最受欢迎。Python的优势在于通用，而R的优势在于统计学家组成的社区。这些统计学家真是高产，也很酷，经常制造出令人惊艳的分析包。**
+ //但是这个文章写与2017.8.29，所以python库更新的能力肯定是越来越强的，今天是2018.7.2

咱们这里就用Python来做数据清理，然后用R做情绪分析，并且把结果可视化输出。

作者：王树义
链接：https://www.jianshu.com/p/0c782715e58a
來源：简书
简书著作权归作者所有，任何形式的转载都请联系作者获得授权并注明出处。

** 我会尽力全部使用python，自己去找相关的库来完成**

但是几乎找不到，还是我太天真，去试水R好了，也不能用中文了，哎

教程给的是权力的游戏的剧本，我用教父的剧本好了

https://www.douban.com/note/513904663/

中文

首先需要清理文本数据，完成以下两个任务：

1. 把与剧情正文无关的内容去除；
2. 将数据转换成R可以直接做情绪分析的结构化数据格式，（我就是转成python可以直接做情绪分析的结构化数据格式）

# 教父

In [1]:
import pandas as pd
import re

In [2]:
with open('theGodFather.txt',encoding='utf8') as f:
    data=f.read()

+ 如果上面使用f.readlines(),则data就成为了list类型，无法直接使用split（）函数。
+ 使用print(data)，数据正确读入，接下来去掉非正文的文本

In [3]:
data=data.split('另一黑帮收买')[1]

+ 解释一下上面这个命令，data是str类型，S.split(sep=None, maxsplit=-1) 返回str列表
+ 返回S中单词的一个使用分隔字符串分隔的列表，如果给出了maxsplit，则最多分隔maxsplit次。
+ 如果没有明确给出分隔符，则任意空白字符都是分隔符，空白字符串将结果中移除。

这里其实就是以‘'另一黑帮收买’为分隔，分为前后两部分，取[1]，也就是后面部分

删除了不属于正文部分的开头部分,可以先用print(data)看一下

然后以同样方式处理结尾部分，但是这个剧本结尾没有多余的废话

**移除了开头和结尾的多余内容后，我们来移除空行。这里我们需要用到正则表达式**


解释下这句代码：也就是把空行用空字符""代替
re.sub(pattern, repl, string, count=0, flags=0)
    
re.sub功能是对于一个输入的字符串，利用正则表达式，来实现字符串替换处理的功能返回处理后的字符串
   
re.sub共有五个参数，三个必选参数 pattern，repl，string，两个可选参数count，flags

https://blog.csdn.net/MHSMIE/article/details/71941328

这个博客写的不错

In [40]:
regex = r'^\s*|\s*$'
subst = ""
data=re.sub(regex, subst, data, 0, re.MULTILINE)
# 由于我使用这个正则一直无效，所以最简单粗暴的就是不用正则，反正都知道要去除的字符长得很规范，直接替换就好了,
# 但是这样替换把该有换行的地方都删掉了，不行，我要的只是去掉空行的换行符
# 后来找到了全角匹配的办法，
# regex = r'$\n[\u0391-\uFFE5]{2}'但是这样之后也是把所有空格都删除了
# 最后，在同一个网站找到上面这个正则，满足只删除空行，但不删除有意义的换行符,即删除了\u3000，和多余的\n，但是有用的换行保留

+ 后来按照这样的正则模式执行之后，发现空行还是没有删除，
+ 直接  data （不是print(data）)
+ 显示出了\n\u3000\u3000 \n\u3000\u3000 这样的字符串，百度发现,\u3000这是中文全角空格，所以我的正则应该由
    + \u3000 是全角的空白符,根据Unicode编码标准及其基本多语言面的定义， \u3000 属于CJK字符的CJK标点符号区块内，是空白字符之一。它的名字是 Ideographic Space ，有人译作表意字空格、象形字空格等。顾名思义，就是全角的 CJK 空格。它跟 nbsp 不一样，是可以被换行间断的。常用于制造缩进， wiki 还说用于抬头，但没见过。

+ \s 空白字符：[ \t\n\x0B\f\r] 
+ \S 非空白字符：[^\s] 


+ regex=r"^$\n" 改为 regex=r"^$\n\n\u3000\u3000"


+ 注意，markdown语法把$$之间的文本括起来了，所以我用了转义，把$弄好

进一步百度，发现：

**在对文本进行处理的时候经常会遇见要对括号和标点进行匹配，常见的英文(半角)符号如( ) 直接用正则匹配即可
，但是遇见全角字符(中文括号、标点)，直接用正则匹配会存在问题：因为编码通常为为utf8，若直接匹配，中文括号的3字节编码会和一些中文的字节编码重复，产生意想不到的结果，若用decode转为unicode编码，则可避免产生错误结果，但也无法直接用正则匹配到**

接下来利用换行符把文本进行分割

In [41]:
lines=data.split('\n')

然后给每一行加上行号，这个可以直接用pandas的DataFrame来做吧

In [43]:
len(lines)

1293

In [42]:
df=pd.DataFrame()

In [44]:
df['line']=range(1,1294)

可以使用df['line']来看看序号是否正确

In [46]:
df['text']=lines

In [47]:
df.head()

,line,text
0,1,1.堂科列奥办公室
1,2,随着片名《教父》的出现，声带上旁白：“我信任美国。”接着，身穿黑色西服、年约六十、神情紧张的...
2,3,波拿塞那：我在美国成了家立了业。（画面后景逐渐出现）我把女儿以美国方式培养成人。我给她自由，...
3,4,此人此时是在“教父”堂科列奥（注1）的办公室内，百叶窗放下了，房间中暗沉沉的。墙上映着百叶窗...
4,5,堂科列奥：波拿塞那，我们是多年的老相识了，可这还是你第一次来求助于我。你最后一次请我上你家去...


所以数据导入正确，格式也正确，为了便于R读取处理，将这个df存为csv文件

但是在网上找了很多，网上大部分都是情感分析，几乎没有针对文本的情绪分析，针对面部表情的情绪分析倒是很多，有些虽然翻译成情绪分析，但是其英语还是sentiment analysis

# 权利的游戏

先照猫画虎再说,稍微看了下R，首先有个R基础安装包（80多M），然后有个集成开发环境RStudio（85M），所以安在台式机上了

因为是网页全选复制，所以除了正文外，开头部分和结尾部分都有些非正文文本，需要处理掉

In [58]:
with open('Game-of-thrones.txt',encoding='utf8') as f:
    content=f.read()

In [59]:
content=content.split('Opening Credits]')[1]

In [60]:
content=content.split('[End Credits')[0]

去除开头和结尾的非正文文本之后，接下来去除空行

In [67]:
regex=r'^$\n'
subst=""
content=re.sub(regex,subst,content,0, re.MULTILINE)

上面的语句，如果不加上 ,0, re.MULTILINE 这两个参数，也无法完成去除空行的效果

+ flags，匹配模式，可以使用按位或‘|‘表示同时生效，也可以在正则表达式中指定。
+ re.I忽略大小写
+ re.L表示特殊字符集\w,\W,\b,\B,\s,\S
+ re.M表示多行模式
+ re.S ‘.’包括换行符在内的任意字符
+ re.U表示特殊字符集\w,\W,\b,\B,\d,\D,\s,\D

发现除了换行符之外，还有一些分隔符，虚线表示的，也进行删除

In [72]:
regex=r'^- +$\n'
subst=""
content=re.sub(regex,subst,content,0, re.MULTILINE)

In [74]:
lines=content.split('\n')

In [76]:
len(lines)

478

In [75]:
df2=pd.DataFrame()

In [77]:
df2['line']=range(1,479)

In [78]:
df2['text']=lines

In [79]:
df2.head()

,line,text
0,1,[First scene shows the location of Casterly Ro...
1,2,CATELYN: Are you sure about this?
2,3,ROBB: No.
3,4,CATELYN: It's dangerous.
4,5,[ROBB nods in agreement]


教程那个人用的是mac，所以有框框，像真的数据表一样，好像还是人家的好看

In [80]:
df2.to_csv('data.csv', index=False)

## R安装

然后到台式机上，试试R

+ 安好基础包和集成开发环境后，按照教程上的，先创建一个R notebook，首先会提示让你进行包的安装，然后安装，类似Python
+ 安装好之后，会有一段文字，这个叫R markdown语法，和这个的很像，
+ 也是一行一行执行的，但是需要选中，不是说光标停留在哪行就执行哪行，而是要选中再执行，才有效
+ R语言可以直接在执行环境里进行安装包，和python一样，安了之后需要导入才可以使用
    + install.packages("dplyr")
    + install.packages("tidytext")
    + install.packages("tidyr")
    + install.packages("ggplot2")
+ 各种执行信息显示在左下角的Console中，数据显示在右边的Script中（前提是用数据，才会显示出Script这个窗口）
+ 导入dplyr包时，会显示很多红色信息，但是这不是错误，注意，不是bug，是自动显示的
    + 执行library语句调用这些包。
    + library(dplyr)
    + library(tidytext)
    + library(tidyr)
    + library(ggplot2)

## 分词

在R里面，可以采用Tidy Text方式来做。执行的语句是unnest_token，我们把原先的句子拆分成为单词。

+ tidy_script <- script %>%
  + unnest_tokens(word, text)
+ head(tidy_script)

所以这里得到分词后的结果是tidy_script


+  line     word
+ 1      1    first
+ 1.1    1    scene
+ 1.2    1    shows
+ 1.3    1      the
+ 1.4    1 location
+ 1.5    1       of

这里原先的行号依然被保留。我们可以看到每一个词来自于哪一行，这有利于下面我们对行甚至段落单位进行分析。

## 找出每个单词的情绪

调用加拿大国家研究委员会发布的情绪词典。这个词典在tidytext包里面内置了，就叫做nrc

+ tidy_script %>%
  + inner_join(get_sentiments("nrc")) %>%
  + arrange(line) %>%
  + head(10)

很明显，这里做了一个等值连接，只是把分词后的结果和NRC表做了内连接，把共同含有的单词找出来了

| |line|word|sentiment|
|:-|:-|
| 1 |    1|         rock|     positive|
| 2 |    1|    ancestral   |     trust|
| 3 |    1|        giant |        fear|
| 4 |    1| representing| anticipation|
| 5 |    1|        stark|     negative|
| 6 |    1|        stark |       trust|
| 7 |    1|        stark  |   negative|
| 8 |    1|        stark  |      trust|
| 9 |    4|    dangerous    |     fear|
|10 |    4|    dangerous    | negative|


可以看到，有的词对应某一种情绪属性，有的词同时对应多种情绪属性。注意**nrc包里面不仅有情绪，而且还有情感（正向和负向）**

这个结果可以在data.csv文件中找到对应的单词，但是很明显，也没有识别出人名，Stark是僵硬的意思，所以这里给出了负面
+ 前8个单词在第一行文本中，第一行中有两个stark,stark这个单词有情绪，trust，也有情感，negative
+ 教程里最后的词是dangerous，我的是father(100行)和horse（101行），dangerous在第4行，

|id|name|
|:-|:-|
|1|A1|
|2|A2|
|3|A3|
Markdown创建表格

## 每行（句）多少个情感词

对单词的情绪已经清楚了。下面我们来综合判断每一行的不同情感分别含有几个词。

    tidy_script %>%
      inner_join(get_sentiments("nrc")) %>%
      count(line, sentiment) %>%
      arrange(line) %>%
      head(10)
Markdown每行缩进四个空格，就是代码块格式了

如果我们以1行为单位分析情感变化，粒度过细。鉴于整个剧本包含了几百行文字，我们以5行作为一个基础单位，来进行分析。

这里我们使用index来把原先的行号处理一下，分成段落。%/%代表整除符号，这样0-4行就成为了第一段落，5-9行成为第二段落，以此类推。

但是这里报错了，即便是Stack Overflow上的答案，也看不懂了，这篇教程就到这吧